# Part 2 - Regression with Image

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = 'cuda'
learning_rate = 0.01
momentum = 0.5
n_epochs = 50

## Prepare MNIST image dataset

### Define Image transform

In [ ]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize(
                                                (0.1307,), (0.3081,))])

### Download torchvision data

In [ ]:
traindata = torchvision.datasets.MNIST('./data', train=True, download=True,
                                       transform=transform)
testdata = torchvision.datasets.MNIST('./data', train=False, download=True,
                                      transform=transform)

### Create dataloader

In [ ]:
batch_size_train = 128
batch_size_test = 32

train_loader = torch.utils.data.DataLoader(traindata, batch_size=batch_size_train, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(testdata, batch_size=batch_size_test, 
                                           shuffle=True)

## Define Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [ ]:
network = Net().to(device)
# optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
optimizer = optim.Adam(network.parameters())

## Train Model

In [ ]:
train_losses, train_counter, test_losses = [], [], []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
def evaluate():
    network.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = network(data)
            result = output.squeeze(dim=1).int().cpu() == target.cpu() # Compare predictions and target
            test_loss += F.mse_loss(output.squeeze(dim=1), target.float(), size_average=False).item()
            correct += np.count_nonzero(result) # Count occurences of true
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, n_epochs + 1):
    log_interval = 100
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = network(data)
        loss = F.mse_loss(output.squeeze(dim=1), target.float())
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(network.state_dict(), 'model.pth')
            torch.save(optimizer.state_dict(), 'optimizer.pth')
    evaluate()

## Test Model

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

with torch.no_grad():
    predictions = network(example_data.to(device))

In [ ]:
fig = plt.figure()
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    plt.title("Predicted: {}".format(predictions.squeeze(dim=1).int().cpu().detach().numpy()[i]))
    plt.xticks([])
    plt.yticks([])
fig

### View model weights

In [ ]:
for name, param in network.state_dict().items():
    print("Layer: ", name)
    print(param)
    print()